# TPOT - Credit Card Fraud Detection (Fixed for TPOT 2.x)

This notebook demonstrates automated machine learning for credit card fraud detection using TPOT 2.x with best practices:
- Fixed for TPOT 2.x API compatibility
- Proper handling of imbalanced data
- Comprehensive evaluation metrics
- Data validation and exploration
- Model comparison and visualization

> ### Install Required Libraries

In [ ]:
# Install TPOT and additional libraries for visualization
!pip install tpot matplotlib seaborn scikit-learn imbalanced-learn

> ### Import Libraries

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TPOT and sklearn
from tpot import TPOTClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    roc_curve,
    precision_recall_curve,
    f1_score,
    accuracy_score,
    precision_score,
    recall_score
)

# For handling imbalanced data
from imblearn.over_sampling import SMOTE
from collections import Counter

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Check TPOT version
import tpot
print(f"TPOT version: {tpot.__version__}")
print("All libraries imported successfully!")

> ### Import Dataset

In [ ]:
# Upload dataset from local drive
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

In [ ]:
# Load the TPOT.csv dataset
dataset = pd.read_csv('TPOT.csv', sep=';', header=None)

# Add descriptive column headers
dataset.columns = [
    'first_time_customer',
    'order_dollar_amount',
    'num_items',
    'age',
    'web_order',
    'total_transactions_to_date',
    'hour_of_day',
    'billing_shipping_zip_equal',
    'reported_as_fraud_historic'
]

print(f"Dataset loaded: {dataset.shape[0]} rows, {dataset.shape[1]} columns")

> ### Exploratory Data Analysis

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
display(dataset.head())

# Dataset information
print("\nDataset Info:")
dataset.info()

# Statistical summary
print("\nStatistical Summary:")
display(dataset.describe())

In [ ]:
# Check for missing values
print("Missing values per column:")
print(dataset.isnull().sum())

# Check data types
print("\nData types:")
print(dataset.dtypes)

> #### Class Balance Analysis (Critical for Fraud Detection!)

In [ ]:
# Analyze class distribution
fraud_counts = dataset['reported_as_fraud_historic'].value_counts()
fraud_ratio = fraud_counts[1] / fraud_counts[0] if len(fraud_counts) > 1 else 0

print("\n" + "="*50)
print("CLASS DISTRIBUTION ANALYSIS")
print("="*50)
print(f"Non-Fraud (0): {fraud_counts.get(0, 0):,} ({fraud_counts.get(0, 0)/len(dataset)*100:.2f}%)")
print(f"Fraud (1):     {fraud_counts.get(1, 0):,} ({fraud_counts.get(1, 0)/len(dataset)*100:.2f}%)")
print(f"Fraud Ratio:   1:{1/fraud_ratio:.1f}" if fraud_ratio > 0 else "No fraud cases found")
print("="*50)

# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
fraud_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Class Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Class (0=Normal, 1=Fraud)')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Normal', 'Fraud'], rotation=0)
for i, v in enumerate(fraud_counts):
    axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')

# Pie chart
axes[1].pie(fraud_counts, labels=['Normal', 'Fraud'], autopct='%1.1f%%',
            colors=['#2ecc71', '#e74c3c'], startangle=90)
axes[1].set_title('Class Distribution (%)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Warning for imbalanced data
if fraud_ratio < 0.1:
    print("\n⚠️  WARNING: Highly imbalanced dataset detected!")
    print("   Consider using SMOTE or adjusting class weights.")

> #### Feature Distribution Visualization

In [ ]:
# Visualize feature distributions by fraud status
numeric_features = [
    'order_dollar_amount', 'num_items', 'age', 
    'total_transactions_to_date', 'hour_of_day'
]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, feature in enumerate(numeric_features):
    for fraud_label in [0, 1]:
        data = dataset[dataset['reported_as_fraud_historic'] == fraud_label][feature]
        axes[idx].hist(data, alpha=0.6, bins=30, 
                      label=f'Fraud={fraud_label}',
                      color='#e74c3c' if fraud_label == 1 else '#2ecc71')
    axes[idx].set_title(f'{feature.replace("_", " ").title()}', fontweight='bold')
    axes[idx].legend()
    axes[idx].set_xlabel(feature.replace('_', ' ').title())
    axes[idx].set_ylabel('Frequency')

# Remove extra subplot
fig.delaxes(axes[-1])

plt.suptitle('Feature Distributions by Fraud Status', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

> #### Correlation Analysis

In [ ]:
# Compute correlation matrix
plt.figure(figsize=(12, 10))
correlation_matrix = dataset.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Show correlations with target variable
print("\nCorrelation with Fraud (Target Variable):")
target_corr = correlation_matrix['reported_as_fraud_historic'].sort_values(ascending=False)
print(target_corr)

> ### Data Preprocessing

In [ ]:
# Separate features and target
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Scale features using MinMaxScaler
print("\nScaling features...")
scaler = preprocessing.MinMaxScaler()
X_scaled = scaler.fit_transform(X)

print("Feature scaling completed!")
print(f"Scaled features range: [{X_scaled.min():.3f}, {X_scaled.max():.3f}]")

> ### Train/Test Split

In [ ]:
# Split dataset: 75% training, 25% testing (standard split)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.25,      # Use 25% for testing
    random_state=42,     # For reproducibility
    stratify=y           # Maintain class distribution in splits
)

print("\n" + "="*50)
print("TRAIN/TEST SPLIT")
print("="*50)
print(f"Training set:   {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"Test set:       {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"\nTraining fraud cases: {sum(y_train)} ({sum(y_train)/len(y_train)*100:.2f}%)")
print(f"Test fraud cases:     {sum(y_test)} ({sum(y_test)/len(y_test)*100:.2f}%)")
print("="*50)

> ### Handle Class Imbalance (Optional but Recommended)

In [ ]:
# Option: Apply SMOTE to balance training data

USE_SMOTE = True  # Set to True to enable SMOTE

if USE_SMOTE:
    print("\nApplying SMOTE to balance training data...")
    print(f"Before SMOTE: {Counter(y_train)}")
    
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
    
    print(f"After SMOTE:  {Counter(y_train_balanced)}")
    print(f"\nTraining samples increased from {len(y_train):,} to {len(y_train_balanced):,}")
    
    # Use balanced data for training
    X_train_final = X_train_balanced
    y_train_final = y_train_balanced
else:
    print("\nUsing original imbalanced training data")
    X_train_final = X_train
    y_train_final = y_train

> ### TPOT Classifier Configuration (TPOT 2.x Compatible)

In [ ]:
# Configuration - Adjust these parameters
GENERATIONS = 5        # Number of pipeline iterations (3-10 recommended)
POPULATION_SIZE = 20   # Number of pipelines per generation (10-50 recommended)
CV_FOLDS = 5           # Cross-validation folds
SCORING = 'roc_auc'    # Scoring metric for fraud detection

print("\n" + "="*50)
print("TPOT CONFIGURATION")
print("="*50)
print(f"Generations:      {GENERATIONS}")
print(f"Population Size:  {POPULATION_SIZE}")
print(f"CV Folds:         {CV_FOLDS}")
print(f"Scoring Metric:   {SCORING}")
print(f"Use SMOTE:        {USE_SMOTE}")
print("="*50)

> ### Run TPOT Training (Fixed for TPOT 2.x)

In [ ]:
# Initialize TPOT Classifier (TPOT 2.x compatible)
print("\nInitializing TPOT Classifier...")

tpot = TPOTClassifier(
    generations=GENERATIONS,
    population_size=POPULATION_SIZE,
    cv=CV_FOLDS,
    scoring=SCORING,
    random_state=42,
    n_jobs=-1,              # Use all CPU cores
    # Note: 'verbosity' is removed - TPOT 2.x uses logging instead
    # Note: 'config_dict' and 'early_stop' may not be available in TPOT 2.x
)

# Record start time
start_time = datetime.now()
print(f"\nTraining started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print("\nThis may take several minutes...")
print("="*50)

# Train TPOT
tpot.fit(X_train_final, y_train_final)

# Record end time
end_time = datetime.now()
training_duration = (end_time - start_time).total_seconds()

print("\n" + "="*50)
print("TRAINING COMPLETED!")
print("="*50)
print(f"Training time: {training_duration:.2f} seconds ({training_duration/60:.2f} minutes)")

> ### Model Evaluation - Comprehensive Metrics

In [ ]:
# Get predictions
y_pred = tpot.predict(X_test)
y_pred_proba = tpot.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\n" + "="*60)
print("MODEL PERFORMANCE METRICS")
print("="*60)
print(f"Accuracy:  {accuracy:.4f}  (Overall correctness)")
print(f"Precision: {precision:.4f}  (Of predicted frauds, how many are actual frauds)")
print(f"Recall:    {recall:.4f}  (Of actual frauds, how many did we catch)")
print(f"F1 Score:  {f1:.4f}  (Harmonic mean of precision and recall)")
print(f"ROC-AUC:   {roc_auc:.4f}  (Area under ROC curve)")
print("="*60)

# Detailed classification report
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred, 
                          target_names=['Normal', 'Fraud'],
                          digits=4))

> #### Confusion Matrix Visualization

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Absolute values
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Normal', 'Fraud'],
            yticklabels=['Normal', 'Fraud'],
            cbar_kws={'label': 'Count'})
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Actual', fontsize=12)
axes[0].set_xlabel('Predicted', fontsize=12)

# Add annotations
axes[0].text(0.5, -0.15, f'True Negatives: {cm[0,0]}', 
             transform=axes[0].transAxes, ha='left', fontsize=10)
axes[0].text(0.5, -0.20, f'False Positives: {cm[0,1]}', 
             transform=axes[0].transAxes, ha='left', fontsize=10, color='red')
axes[0].text(0.5, -0.25, f'False Negatives: {cm[1,0]}', 
             transform=axes[0].transAxes, ha='left', fontsize=10, color='red')
axes[0].text(0.5, -0.30, f'True Positives: {cm[1,1]}', 
             transform=axes[0].transAxes, ha='left', fontsize=10)

# Percentage values
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
sns.heatmap(cm_percent, annot=True, fmt='.2f', cmap='Greens', ax=axes[1],
            xticklabels=['Normal', 'Fraud'],
            yticklabels=['Normal', 'Fraud'],
            cbar_kws={'label': 'Percentage'})
axes[1].set_title('Confusion Matrix (Percentages)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Actual', fontsize=12)
axes[1].set_xlabel('Predicted', fontsize=12)

plt.tight_layout()
plt.show()

> #### ROC and Precision-Recall Curves

In [ ]:
# Calculate curves
fpr, tpr, roc_thresholds = roc_curve(y_test, y_pred_proba)
precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_test, y_pred_proba)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROC Curve
axes[0].plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
axes[0].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curve', fontsize=14, fontweight='bold')
axes[0].legend(loc='lower right', fontsize=11)
axes[0].grid(True, alpha=0.3)

# Precision-Recall Curve
axes[1].plot(recall_curve, precision_curve, linewidth=2, 
             label=f'PR Curve (AUC = {np.trapz(precision_curve, recall_curve):.4f})')
axes[1].axhline(y=sum(y_test)/len(y_test), color='k', linestyle='--', 
                linewidth=1, label='Baseline (Random)')
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curve', fontsize=14, fontweight='bold')
axes[1].legend(loc='lower left', fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 For fraud detection:")
print("   - High RECALL is critical (catch as many frauds as possible)")
print("   - Balance with PRECISION to avoid too many false alarms")
print("   - ROC-AUC shows overall discriminative ability")

> #### Best Pipeline Details

In [ ]:
print("\n" + "="*60)
print("BEST PIPELINE FOUND BY TPOT")
print("="*60)
print(tpot.fitted_pipeline_)
print("="*60)

# Get pipeline steps
print("\nPipeline Steps:")
for idx, (name, transformer) in enumerate(tpot.fitted_pipeline_.steps, 1):
    print(f"{idx}. {name}: {transformer.__class__.__name__}")

> ### Export Results

> #### Export Pipeline as Python Script

In [ ]:
# Export the best pipeline as a Python script
pipeline_filename = f'tpot_fraud_pipeline_{datetime.now().strftime("%Y%m%d_%H%M%S")}.py'
tpot.export(pipeline_filename)

print(f"✅ Pipeline exported to: {pipeline_filename}")

# Download file (for Google Colab)
from google.colab import files
files.download(pipeline_filename)
print(f"📥 Downloaded: {pipeline_filename}")

> ### Generate Summary Report

In [ ]:
# Create comprehensive summary report
report = f"""
{'='*70}
TPOT CREDIT CARD FRAUD DETECTION - SUMMARY REPORT
{'='*70}

Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

DATASET INFORMATION
{'-'*70}
Total Samples:        {len(dataset):,}
Training Samples:     {len(X_train_final):,}
Test Samples:         {len(X_test):,}
Number of Features:   {X_train.shape[1]}
Fraud Cases (Train):  {sum(y_train_final)} ({sum(y_train_final)/len(y_train_final)*100:.2f}%)
Fraud Cases (Test):   {sum(y_test)} ({sum(y_test)/len(y_test)*100:.2f}%)
SMOTE Applied:        {USE_SMOTE}

TPOT CONFIGURATION
{'-'*70}
Generations:          {GENERATIONS}
Population Size:      {POPULATION_SIZE}
CV Folds:             {CV_FOLDS}
Scoring Metric:       {SCORING}
Training Time:        {training_duration:.2f} seconds ({training_duration/60:.2f} minutes)

MODEL PERFORMANCE
{'-'*70}
Accuracy:             {accuracy:.4f}
Precision:            {precision:.4f}
Recall:               {recall:.4f}
F1 Score:             {f1:.4f}
ROC-AUC:              {roc_auc:.4f}

CONFUSION MATRIX
{'-'*70}
True Negatives:       {cm[0,0]:,}
False Positives:      {cm[0,1]:,}
False Negatives:      {cm[1,0]:,}
True Positives:       {cm[1,1]:,}

BEST PIPELINE
{'-'*70}
{tpot.fitted_pipeline_}

KEY INSIGHTS
{'-'*70}
✓ The model detected {cm[1,1]} out of {sum(y_test)} fraud cases
✓ Missed {cm[1,0]} fraud cases (False Negatives)
✓ Flagged {cm[0,1]} normal transactions as fraud (False Positives)
✓ Overall detection rate: {recall*100:.1f}%

{'='*70}
"""

print(report)

# Save report to file
report_filename = f'tpot_fraud_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'
with open(report_filename, 'w') as f:
    f.write(report)

print(f"\n✅ Report saved to: {report_filename}")
files.download(report_filename)
print(f"📥 Downloaded: {report_filename}")

> ### Save Model (Optional)

In [ ]:
# Save the trained model for later use
import joblib

model_filename = f'tpot_fraud_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pkl'
joblib.dump(tpot.fitted_pipeline_, model_filename)

print(f"✅ Model saved to: {model_filename}")
files.download(model_filename)
print(f"📥 Downloaded: {model_filename}")

print("\n💡 To load the model later:")
print("   loaded_model = joblib.load('model_filename.pkl')")
print("   predictions = loaded_model.predict(X_new)")